In [175]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pickle


In [176]:
df_book = pd.read_csv("data/Books.csv")
df_book.head(3)

/tmp/ipykernel_74712/2249489971.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_book = pd.read_csv("data/Books.csv")


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


In [177]:
df_book.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [178]:
df_book = df_book.drop(['Publisher','Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1)

In [179]:
df_book.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication
0,0195153448,Classical Mythology,Mark P. O. Morford,2002
1,0002005018,Clara Callan,Richard Bruce Wright,2001
2,0060973129,Decision in Normandy,Carlo D'Este,1991


In [180]:
ratings =pd.read_csv("data/Ratings.csv")
ratings.head(3)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


In [181]:
df_book.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
dtype: int64

In [182]:

df_book['Book-Author'] = df_book['Book-Author'].fillna('Unknown Author')

df_book['Year-Of-Publication'] = pd.to_numeric(df_book['Year-Of-Publication'], errors='coerce')
df_book = df_book.dropna(subset=['Year-Of-Publication'])
df_book = df_book[df_book['Year-Of-Publication'] > 1900] 

In [183]:
df_book.isnull().sum()


ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
dtype: int64

In [184]:
merged_df = pd.merge(df_book,ratings,on='ISBN')


In [185]:
merged_df.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001.0,11400,0


In [186]:
combine_book_rating = merged_df.dropna(axis = 0, subset = ['Book-Title'])
book_ratingCount = (combine_book_rating.
     groupby(by = ['Book-Title'])['Book-Rating'].
     count().
     reset_index().
     rename(columns = {'Book-Rating': 'totalRatingCount'})
     [['Book-Title', 'totalRatingCount']]
    )
book_ratingCount.head()

,Book-Title,totalRatingCount
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [187]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'Book-Title', right_on = 'Book-Title', how = 'left')
rating_with_totalRatingCount.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,User-ID,Book-Rating,totalRatingCount
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,2,0,2
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,8,5,14
2,0002005018,Clara Callan,Richard Bruce Wright,2001.0,11400,0,14
3,0002005018,Clara Callan,Richard Bruce Wright,2001.0,11676,8,14
4,0002005018,Clara Callan,Richard Bruce Wright,2001.0,41385,0,14


In [188]:
popularity_threshold = 50
rating_popular_book= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')

In [189]:
rating_popular_book.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,User-ID,Book-Rating,totalRatingCount
30,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,8,0,311
31,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,11676,9,311
32,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,29526,9,311
33,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,36836,0,311
34,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,46398,9,311


In [190]:
rating_popular_book.shape

(285627, 7)

In [191]:
book_features_df=rating_popular_book.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating').fillna(0)
book_features_df.head()

User-ID,8,9,14,16,17,19,23,26,32,39,...,278820,278824,278828,278832,278836,278843,278844,278846,278851,278854
Book-Title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [196]:
from scipy.sparse import csr_matrix

book_features_df_matrix = csr_matrix(book_features_df.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(book_features_df_matrix)

,n_neighbors,5
,radius,1.0
,algorithm,'brute'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [207]:
query_index = np.random.choice(book_features_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(book_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

727


In [202]:
book_features_df.head()

User-ID,8,9,14,16,17,19,23,26,32,39,...,278820,278824,278828,278832,278836,278843,278844,278846,278851,278854
Book-Title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [208]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(book_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, book_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Gerald's Game:

1: Four Past Midnight, with distance of 0.7452007690105594:
2: Rose Madder, with distance of 0.7455445762435664:
3: The Dark Half, with distance of 0.7550762915106664:
4: Carrie, with distance of 0.7555499555586356:
5: Dolores Claiborne, with distance of 0.7580789084074817:
